In [2]:
import pandas as pd
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
#data import
train = pd.read_csv('./Data/TitanicTrain.csv')
test = pd.read_csv('./Data/TitanicTest.csv')
gender_submission = pd.read_csv('./Data/TitanicGender_submission.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Clean
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,1
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,0


In [8]:
# numeric
train['Sex_Clean'] = train['Sex'].astype('category').cat.codes
test['Sex_Clean'] = test['Sex'].astype('category').cat.codes
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sex_Clean
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [20]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Sex_Clean        0
dtype: int64

In [16]:
# Embarked = 승객이 어느 항구에서 탔는지 / NaN값 처리 ( 가장 많은 S값으로 처리 )
train["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [21]:
train['Embarked'].fillna('S', inplace = True)

In [24]:
train['Embarked'].isnull().sum()

0

In [25]:
# numeric
train['Embarked_clean'] = train['Embarked'].astype('category').cat.codes
test['Embarked_clean'] = test['Embarked'].astype('category').cat.codes

In [28]:
# sibsp / of siblings / spouses aboard the Titanic
# parch / of parents / children aboard the Titanic

# Family
train['Family'] = 1 + train['SibSp'] + train['Parch']
test['Family'] = 1 + test['SibSp'] + test['Parch']

# Solo
train['Solo'] = (train['Family'] == 1)
test['Solo'] = (test['Family'] == 1)

In [31]:
# Binning         /      5구간으로 나누기
train['FareBin'] = pd.qcut(train['Fare'], 5)
test['FareBin'] = pd.qcut(test['Fare'], 5)

train['FareBin'].value_counts()

(7.854, 10.5]        184
(21.679, 39.688]     180
(-0.001, 7.854]      179
(39.688, 512.329]    176
(10.5, 21.679]       172
Name: FareBin, dtype: int64

In [38]:
train['FareClean'] = train['FareBin'].astype('category').cat.codes
test['FareClean'] = test['FareBin'].astype('category').cat.codes

train['FareClean'].value_counts().sort_index()

0    179
1    184
2    172
3    180
4    176
Name: FareClean, dtype: int64

In [39]:
train['Title'] = train['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)
test['Title'] = test['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [40]:
train['Title'].value_counts()

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Mme           1
Countess      1
Sir           1
Jonkheer      1
Ms            1
Capt          1
Don           1
Lady          1
Name: Title, dtype: int64

In [41]:
# reduction in complexity

train['Title'] = train['Title'].replace(['Countess','Capt', 'Lady','Don','Dona', 'Major','Col', 'Rev', 'Sir', 'Jonkheer', 'Dr'], 'Other')
train['Title'].value_counts()

Mr        517
Miss      182
Mrs       125
Master     40
Other      23
Mlle        2
Ms          1
Mme         1
Name: Title, dtype: int64

In [42]:
train['Title'] = train['Title'].replace('Mlle', 'Miss')
train['Title'] = train['Title'].replace('Ms', 'Miss')
train['Title'] = train['Title'].replace('Mme', 'Mrs')

In [43]:
train['Title'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Other      23
Name: Title, dtype: int64

In [44]:
test['Title'] = test['Title'].replace(['Lady', 'Countess', 'Jonkheer', 'Col', 'Capt', 'Dr', 'Don', 'Rev', 'Major', 'Sir', 'Dona'], 'Other')
test['Title'] = test['Title'].replace('Mlle', 'Miss')
test['Title'] = test['Title'].replace('Ms', 'Miss')
test['Title'] = test['Title'].replace('Mme', 'Mrs')

test['Title'].value_counts()

Mr        240
Miss       79
Mrs        72
Master     21
Other       6
Name: Title, dtype: int64

In [55]:
train['TitleClean'] = train['Title'].astype('category').cat.codes
test['TitleClean'] = test['Title'].astype('category').cat.codes

In [95]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,...,Embarked_clean,Family,Solo,FareBin,FareClean,Title,Title_clean,TitleClean,Age_clean,Cabin_clean
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,...,1,1,True,"(7.796, 11.025]",1,Mr,2,2,5.0,5.0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,...,2,2,False,"(-0.001, 7.796]",0,Mrs,3,3,8.0,5.0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,...,1,1,True,"(7.796, 11.025]",1,Mr,2,2,10.0,5.0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,...,2,1,True,"(7.796, 11.025]",1,Mr,2,2,4.0,5.0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,...,2,3,False,"(11.025, 21.438]",2,Mrs,3,3,3.0,5.0


In [66]:
# AgeNull = 중간값
train['Age'].fillna(train.groupby("Title")['Age'].transform("median"), inplace=True)
test['Age'].fillna(test.groupby("Title")['Age'].transform("median"), inplace=True)
print(train['Age'].max(), train['Age'].min())

80.0 0.42


In [67]:
train.loc[ train['Age'] <= 10, 'Age_clean'] = 0
train.loc[(train['Age'] > 10) & (train['Age'] <= 16), 'Age_clean'] = 1 # 11 ~ 16 / 5
train.loc[(train['Age'] > 16) & (train['Age'] <= 20), 'Age_clean'] = 2
train.loc[(train['Age'] > 20) & (train['Age'] <= 26), 'Age_clean'] = 3
train.loc[(train['Age'] > 26) & (train['Age'] <= 30), 'Age_clean'] = 4
train.loc[(train['Age'] > 30) & (train['Age'] <= 36), 'Age_clean'] = 5
train.loc[(train['Age'] > 36) & (train['Age'] <= 40), 'Age_clean'] = 6
train.loc[(train['Age'] > 40) & (train['Age'] <= 46), 'Age_clean'] = 7
train.loc[(train['Age'] > 46) & (train['Age'] <= 50), 'Age_clean'] = 8
train.loc[(train['Age'] > 50) & (train['Age'] <= 60), 'Age_clean'] = 9 # 51 ~ 60 / 10
train.loc[ train['Age'] > 60, 'Age_clean'] = 10

In [68]:
test.loc[ test['Age'] <= 10, 'Age_clean'] = 0
test.loc[(test['Age'] > 10) & (test['Age'] <= 16), 'Age_clean'] = 1
test.loc[(test['Age'] > 16) & (test['Age'] <= 20), 'Age_clean'] = 2
test.loc[(test['Age'] > 20) & (test['Age'] <= 26), 'Age_clean'] = 3
test.loc[(test['Age'] > 26) & (test['Age'] <= 30), 'Age_clean'] = 4
test.loc[(test['Age'] > 30) & (test['Age'] <= 36), 'Age_clean'] = 5
test.loc[(test['Age'] > 36) & (test['Age'] <= 40), 'Age_clean'] = 6
test.loc[(test['Age'] > 40) & (test['Age'] <= 46), 'Age_clean'] = 7
test.loc[(test['Age'] > 46) & (test['Age'] <= 50), 'Age_clean'] = 8
test.loc[(test['Age'] > 50) & (test['Age'] <= 60), 'Age_clean'] = 9
test.loc[ test['Age'] > 60, 'Age_clean'] = 10

In [76]:
train['Cabin'].value_counts()

G6             4
C23 C25 C27    4
B96 B98        4
F33            3
F2             3
              ..
C91            1
B42            1
B86            1
D10 D12        1
C7             1
Name: Cabin, Length: 146, dtype: int64

In [72]:
train['Cabin'].str[:1].value_counts() # ex.) E46

C    57
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

In [80]:
# Pclass = Ticket class / 1 = 1st, 2 = 2nd, 3 = 3rd
train['Pclass']

5      3
6      1
7      3
8      3
9      2
      ..
886    2
887    1
888    3
889    1
890    3
Name: Pclass, Length: 886, dtype: int64

In [81]:
mapping = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4,
    'F': 5,
    'G': 6,
    'T': 7
}

train['Cabin_clean'] = train['Cabin'].str[:1]
train['Cabin_clean'] = train['Cabin_clean'].map(mapping) # A = 0, B = 1
train['Cabin_clean'] = train.groupby('Pclass')['Cabin_clean'].transform('median')

test['Cabin_clean'] = test['Cabin'].str[:1]
test['Cabin_clean'] = test['Cabin_clean'].map(mapping)
test['Cabin_clean'] = test.groupby('Pclass')['Cabin_clean'].transform('median')

train['Cabin_clean'].value_counts()

5.0    488
2.0    214
4.5    184
Name: Cabin_clean, dtype: int64

In [82]:
test['Cabin_clean'].value_counts()

5.0    311
2.0    107
Name: Cabin_clean, dtype: int64

In [88]:
train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'Sex_Clean',
       'Embarked_clean', 'Family', 'Solo', 'FareBin', 'FareClean', 'Title',
       'TitleClean', 'Age_clean', 'Cabin_clean'],
      dtype='object')

In [96]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'Sex_Clean', 'Embarked_clean',
       'Family', 'Solo', 'FareBin', 'FareClean', 'Title', 'Title_clean',
       'TitleClean', 'Age_clean', 'Cabin_clean'],
      dtype='object')

In [97]:
feature = [
    'Pclass',
    'SibSp',
    'Parch',
    'Sex_Clean',
    'Embarked_clean',
    'Family',
    'Solo',
    'TitleClean',
    'Age_clean',
    'Cabin_clean',
    'FareClean',
]

label = [
    'Survived',
]

In [93]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [98]:
data = train[feature]
target = train[label]

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

clf = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=0)
cross_val_score(clf, data, target, cv=k_fold, scoring='accuracy', ).mean()

0.8285240040858017

In [99]:
x_train = train[feature]
x_test = test[feature]
y_train = train[label]

clf = RandomForestClassifier(n_estimators=50, max_depth=6, random_state=0)
clf.fit(x_train, y_train)
gender_submission['Survived'] = clf.predict(x_test)
gender_submission.to_csv('Result.csv',index=False)